In [ ]:
!pip install --upgrade pip
!pip install opencv-python==4.1.2.30
!pip install opencv-python-headless==4.1.2.30 
!pip install pydicom
!pip install easyocr
!pip install Pillow==9.0.0
!pip install imgaug
!pip install tqdm

In [ ]:
from heapq import nlargest
from scipy import pi, sqrt
from scipy.special import hyp2f1
from tqdm import tqdm
from IPython.display import Image,display
from ipywidgets import IntProgress
import cv2
import numpy as np
import pydicom
import easyocr
from matplotlib import pyplot as plt
import pandas as pd
import csv
from shutil import copyfile
from PIL import Image
from PIL import ImageFont, ImageDraw
from heapq import nlargest
from scipy import pi, sqrt
from scipy.special import hyp2f1
from tqdm import tqdm
from IPython.display import Image,display
from ipywidgets import IntProgress
reader = easyocr.Reader(['en'])

## **Extract bio measures from the image**





In [ ]:
def convertMMtoCM(value,unit):
  if(unit=="mm"):
    return str(round(value/10,2))+"cm"
  else:
    return str(round(value,2))+str(unit)

def correctCharacters(text):
  """
  corrects characters that has been mislabeled by the OCR
  """
  text=text.replace("A","4")
  text=text.replace("o","0")
  text=text.replace("O","0")
  text=text.replace("i","1")
  text=text.replace("b","6")
  text=text.replace("B","8")
  text=text.replace(" ",".") #to replace spaces with dots in cases like these : 7 74cm --> 7.74cm
  text=text.replace("#","0")
  return text


class findTheLength :
    def __init__(self,bounds , n):
        self.bounds = bounds
        self.n = n

    def findTheLengthValue(self): 
        if(self.n==0):
            if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n][1])
            elif(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n+1][1])
            elif(any(metric in self.bounds[self.n+2][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n+2][1])
            return 1
        elif(self.n==len(self.bounds)-1):
          if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-1][1])
          return 1
        elif(self.n==len(self.bounds)-2):
          if(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+1][1])
          elif(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-1][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          return 1
        else:
          if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+1][1])
          elif(any(metric in self.bounds[self.n+2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+2][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return (self.bounds[self.n-1][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          return 1
              
def convertTextToFloat(word):
  for i,CARACTER in enumerate(word):
      space=False
      if(CARACTER=='c' or CARACTER=='m'):
        try:
          if(word[i+1]=="m"):
            UNIT=CARACTER+word[i+1]
            if(word[i-1]==" "):
              space=True
              newWORD=word[:i-1]+word[i:]
            else:
              newWORD=word
            for j in range(-7,-3):
              try:
                if(space):
                  if(i+j<0):
                    continue
                  return float(correctCharacters(newWORD[i+j:i-1])),UNIT
                else:
                  if(i+j<0):
                    continue
                  return float(correctCharacters(newWORD[i+j:i])),UNIT
              except:
                continue
        except:
          continue

        
class BioInfo :
    def __init__(self , bounds):
        self.bounds = bounds

    def extractingTheMetrics(self ,bounds) :
      BIO=[None,None,None,None]
      for n,i in enumerate(bounds):
                  
          if (any(metric in i[1] for metric in ["BIP","A BIP","ABIP","BIPI","BlP","A BlP","ABlP","8IP","8lP","A8IP","A8lP","A 8IP"])):
            if(i[1] in ["BIP","A BIP","ABIP","BIPI","BlP","A BlP","ABlP","8IP","8lP","A8IP","A8lP","A 8IP"]):
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:

                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[3] = convertMMtoCM(VALUE, UNIT)

          if (any(metric in i[1] for metric in ["CT","A CT","ACT",'ct',"Ct","cT"])):
            if(i[1] in ["CT","A CT","ACT",'ct',"Ct","cT"]):
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:
                      bio = bio.replace(',','.')
                      bio = bio.replace('t','')
                      bio = bio.replace('u','')
                      bio = bio.replace('s','')
                      bio = bio.replace("n","m")
                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[2] = convertMMtoCM(VALUE, UNIT)

          if (any(metric in i[1] for metric in ["LF",'lf','Lf','lF',"A LF","ALF","alf","Alf","aLf","alF",'LLF','llf','lff'])):
            if(i[1] in ["LF",'lf','Lf','lF',"A LF","ALF","alf","Alf","aLf","alF",'LLF','llf','lff'] ):
              
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:
                      bio = bio.replace("n", "m")
                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[1] = convertMMtoCM(VALUE, UNIT)

          if(any(metric in i[1] for metric in ["CA","A CA","ACA","ca","Ca","cA"])):
            if(i[1] in ["CA","A CA","ACA","ca","Ca","cA"]):
              FTL = findTheLength(self.bounds,n)
              bio=FTL.findTheLengthValue()

              if(bio!=1):
                try:
                    bio = bio.replace("n", "m")
                    VALUE,UNIT=convertTextToFloat(bio)
                except:
                    continue
                BIO[0]=convertMMtoCM(VALUE,UNIT)

      return BIO



## **Preprocess images to get rid of calipers and clean the image then get a mask**

In [ ]:
def findBiggestContour(image):
  #Find the biggest contour in the image , using findContours function
    import easyocr
    try:
      img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except :
      img=image
 
    ret, thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = []
    for c in contours:
        area = cv2.contourArea(c)
        areas.append(area)
          
    biggest_Contour = []
    if len(areas) > 1:
        max_value = np.partition(areas, -2)[-1]
        max_index = areas.index(max_value)
        biggest_Contour = contours[max_index]
    
    return biggest_Contour


def cropContour(originalImage, biggestContour):
  # Crop the biggest contour of image , the size of the image will change
  try:
    rect = cv2.boundingRect(biggestContour)
    x,y,w,h = rect
    if (x==0 and y ==0):
      croped = originalImage[y:y+w,x:x+h].copy()
    else:
      croped = originalImage[y:y+h, x:x+w].copy()
    contour = biggestContour - biggestContour.min(axis=0)
    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1, cv2.LINE_AA)
    dst = cv2.bitwise_and(croped, croped, mask=mask)
    return dst
  except:
    return 0


def paddingSidesOfMask(mask):
  #basically dilate the output masks.
  newmask=mask.copy()
  for y,vy in enumerate(mask):
      for x,vx in enumerate(vy):
        if(mask[y,x]==255):
          # checks if the value of the pixel is equal to 255, if it is, we get all points that’s around that specific pixel
          newpoints=getNeighborsPoints([y,x])
          for j in newpoints:
            try:
              newmask[j[0],j[1]]=255
            except:
              continue
  return newmask


def getNeighborsPoints(point):
  #get all points that’s around that specific pixel
  liste=[]
  liste.append([point[0]-1,point[1]])
  liste.append([point[0]-1,point[1]-1])
  liste.append([point[0]-1,point[1]+1])
  liste.append([point[0]+1,point[1]])
  liste.append([point[0]+1,point[1]-1])
  liste.append([point[0]+1,point[1]+1])
  liste.append([point[0],point[1]-1])
  liste.append([point[0],point[1]+1])
  return liste



def getCenter(contours):
  """
  gets the center of the contour
  input : contours
  output : float
  """
  M = cv2.moments(contours)
  try:
    cX = int(M['m10'] /M['m00'])
    cY = int(M['m01'] /M['m00'])
    return [cX,cY]
  except:
    return [0,0]

def getAverage(contours):
  """
  gets the average of all the average distances of every point (to compare the outlier point to this average)
  input : contours
  output : float
  """
  avgavg=[]
  for j in contours:
    avgdistance=[]
    centerj=getCenter(j)
    for i in contours:
      centeri=getCenter(i)
      dx= centeri[0] - centerj[0]
      dy = centeri[1] - centerj[1]
      D = np.sqrt(dx*dx+dy*dy)
      avgdistance.append(D)
    avgavg.append((sum(avgdistance)/len(avgdistance)))
    if len(avgavg)!=0:
      return (sum(avgavg)/len(avgavg))
    else:
      return 1


def centerOfBound(bound):
  # get the center of the text
  p0, p1, p2, p3 = bound[0]
  xcenter=(p0[0]+p2[0])/2
  ycenter=(p0[1]+p2[1])/2
  return [int(xcenter),int(ycenter)]

def getCloseBoxes(point,bounds):
  centers=[]
  for i in bounds:
    # we parse through the entire texts extracted, we get the center of the text
      centers.append(centerOfBound(i))
  close_boxes=[]
  thelist=[]
  thelist.append(point)
  for index1,i in enumerate(thelist):
      for index2,j in enumerate(centers):
        dx= i[0] - j[0]
        dy = i[1] - j[1]
        D = np.sqrt(dx*dx+dy*dy)
        if(D!=0 and D<140 and  (j not in thelist)):
          thelist.append(j)
          close_boxes.append(bounds[index2][0])
  return close_boxes,thelist

def getBigBox(usefulbounds):
  """
  Determines the position of where the biggest x and y coordinates out of all the rectangles 
  which will be the bottom right corner of our big rectangle, 
  and in the same fashion, the smallest x and y coordinates that will be   
  considered the top left corner of the big rectangle, 
  and we just return these 2 coordinates of top left and bottom right corners.

  """
  twopoint=[]
  xsmall=500000
  ysmall=500000
  xbig=0
  ybig=0
  for i in usefulbounds:
    for j in i:
      if(j[0]>xbig):
        xbig=j[0]
      if(j[0]<xsmall):
        xsmall=j[0]
      if(j[1]>ybig):
        ybig=j[1]
      if(j[1]<ysmall):
        ysmall=j[1]
  return (xsmall,ysmall),(xbig,ybig)

def drawBlackBoxes(image,reader):
    """
    it hides text (chunks) with a black box
    input : image and OCRreader
    output : image (cleaned from text)
    """
    bounds = reader.readtext(image)
    alreadyTreatedList=[]
    for ii in bounds:
      i=centerOfBound(ii)
      if(i not in alreadyTreatedList):
        usefullist,thelist=getCloseBoxes(i,bounds)
        usefullist.append(ii[0])
        if(len(usefullist)>=2):
          small,big=getBigBox(usefullist)
          cv2.rectangle(image,(int(small[0]-6),int(small[1]-6)),(int(big[0]+5),int(big[1]+5)),(0,0,0),-1)
          for j in thelist:
            alreadyTreatedList.append(j)
    return image


def removeTextBoxes(image, reader):
    """
    removes text boxes incase they are inside of the US image using OCRreader
    input : image and OCRreader
    output : image (without text)
    """
    image = drawBlackBoxes(image, reader)
    image = cv2.rectangle(image, (0, 0), (image.shape[1], 8), (0, 0, 0), -1)
    return image


def removeOutliers(contours,bin):
  """
  Removes outlier points that aren't part of the ellipse's points
  input: contours, and a parameter()
  output : contours
  """
  actualcontours=[]
  average=getAverage(contours)
  for j in contours:
    avgdistance=[]
    centerj=getCenter(j)
    for i in contours:
      centeri=getCenter(i)
      dx= centeri[0] - centerj[0]
      dy = centeri[1] - centerj[1]
      D = np.sqrt(dx*dx+dy*dy)
      avgdistance.append(D)
    if((sum(avgdistance)/len(avgdistance))<average+bin): actualcontours.append(j)
  return actualcontours

def contoursConvexHull(contours):
    """
    makes a polygon mask out of convex points only
    input : contours (points)
    output : mask
    """
    pts = []
    for i in range(0, len(contours)):
        for j in range(0, len(contours[i])):
            pts.append(contours[i][j])
    pts = np.array(pts)
    result = cv2.convexHull(pts)
    return result

def convertCalipersToMask(original):
    """
    Converts the calipers to a mask using the caliper colors

    input : original image (with calipers only(Text and sides cleaned/cropped))
    output : mask of the calipers
    """
    # Firstly, puts rectangles on the top and leftside, since there's usually a scale on the left, and to make sure there's no top(anonymization)
    cv2.rectangle(original, (0, 0), (1200, 10), (0, 0, 0), -1)
    cv2.rectangle(original, (0, 0), (47, 1200), (0, 0, 0), -1)
    # Secondly, locate the caliper color using HSV color range, and use inRange to select only that color (link to help : https://i.stack.imgur.com/gyuw4.png)
    image = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)
    low_yellow = np.array([85, 0, 20], dtype="uint8")
    hight_yellow = np.array([95, 255, 255], dtype="uint8")
    mask_yellow = cv2.inRange(image, low_yellow, hight_yellow)
    
    low_green = np.array([25, 52, 72])
    high_green = np.array([102, 255, 255])
    mask_green=cv2.inRange(image,low_green,high_green)
    ############
    low_white = np.array([0,0,255-15])
    high_white = np.array([255,15,255])
    mask_white=cv2.inRange(image,low_white,high_white)
    
    if(mask_yellow.any()):
      
      # Using Gaussian and Canny filters to make it easy to get the contours
      blurred = cv2.GaussianBlur(mask_yellow, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = removeOutliers(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:
        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty   

    if(mask_white.any()):
      blurred = cv2.GaussianBlur(mask_white, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = removeOutliers(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:
        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty  
    
    if(mask_green.any()):
      blurred = cv2.GaussianBlur(mask_green, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = removeOutliers(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:
        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty 
    


def imageCropping(image):
    """
    finds the biggest contour (US part only) and crops the rest
    input : image
    output : image (withoutsides)
    """
    biggest_contour = findBiggestContour(image)
    imageCropped = cropContour(image, biggest_contour)
    return imageCropped
  
def cleaningannotations(image):
    """
    detects where the calipers are and does inpaint process to remove it
    input : image
    output : image (cleaned)
    
    """
    ########
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    lower_yellow=np.array([70,0,20], dtype="uint8")
    higher_yellow=np.array([100,255,255], dtype="uint8")
    mask_yellow=cv2.inRange(hsv,lower_yellow,higher_yellow)

    
    ########
    low_green = np.array([25, 52, 72])
    high_green = np.array([102, 255, 255])
    mask_green=cv2.inRange(hsv,low_green,high_green)
    ############
    low_white = np.array([0,0,255-15])
    high_white = np.array([255,15,255])
    mask_white=cv2.inRange(hsv,low_white,high_white)
    
    if (mask_yellow.any()):
      image= cv2.inpaint(image,paddingSidesOfMask(mask_yellow),10,cv2.INPAINT_NS)
      return image
    
    if (mask_green.any()):
      image = cv2.inpaint(image,paddingSidesOfMask(mask_green),10,cv2.INPAINT_NS)
      return image
    
    if(mask_white.any()):
      image = cv2.inpaint(image,paddingSidesOfMask(mask_white),10,cv2.INPAINT_NS)
      return image

    

def getCleanedImageandMask(pixel_array_numpy,reader):
  
    croppedimage=imageCropping(pixel_array_numpy) 
    imageWithRemovedTextBoxes=removeTextBoxes(croppedimage,reader)
    if(croppedimage.shape[0]<600 and croppedimage.shape[1]<800):
      imageWithRemovedTextBoxes = cv2.resize(imageWithRemovedTextBoxes, (800,600), interpolation = cv2.INTER_AREA)
      mask=convertCalipersToMask(imageWithRemovedTextBoxes)
    else:
      mask=convertCalipersToMask(imageWithRemovedTextBoxes)
    cleanedimage =cleaningannotations(imageWithRemovedTextBoxes)
  
      
    return imageWithRemovedTextBoxes,mask,cleanedimage

  
import cv2
import numpy as np
import pydicom
import easyocr
from matplotlib import pyplot as plt
import pandas as pd
import csv
from shutil import copyfile
from PIL import Image
def drawMaskOfimage(image):
  img = image.copy()
  if(len(image.shape)<3):
     img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  croppedimage=imageCropping(img)
  #imageWithRemovedTextBoxes=removeTextBoxes(croppedimage,reader)
  
  # Secondly, locate the caliper color using HSV color range, and use inRange to select only that color (link to help : https://i.stack.imgur.com/gyuw4.png)
  image = cv2.cvtColor(croppedimage, cv2.COLOR_BGR2HSV)
  low_yellow = np.array([85, 0, 20], dtype="uint8")
  hight_yellow = np.array([95, 255, 255], dtype="uint8")
  mask_yellow = cv2.inRange(image, low_yellow, hight_yellow)

  low_green = np.array([25, 52, 72])
  high_green = np.array([102, 255, 255])
  mask_green=cv2.inRange(image,low_green,high_green)
  
  ############
  
  low_white = np.array([0,0,255-15])
  high_white = np.array([255,15,255])
  mask_white=cv2.inRange(image,low_white,high_white)
  
  if(mask_white.any()):
    
    blurred = cv2.GaussianBlur(mask_white, (5, 5), 0)
    imageCanny = cv2.Canny(blurred, 0, 100, 0)
    ret,th = cv2.threshold(imageCanny,127,255, 0)

    #--- Find all the contours in the binary image ---
    contours,hierarchy = cv2.findContours(th,3,1)
    cnt = contours
    big_contour = []
    m = 0
    
    for i in cnt:

      area = cv2.contourArea(i) #--- find the contour having biggest area ---
      if(area > m):
        m = area
        big_contour = i 

    final = cv2.drawContours(imageCanny, big_contour, -1, (0,255,0), 3)



    thresh = cv2.threshold(final, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=2)


    # Find contours, filter using contour threshold area, draw ellipse
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    listofareas =[]
    for c in cnts:
        area = cv2.contourArea(c)
        listofareas.append(area)

    for c in cnts:
        area = cv2.contourArea(c)

        if area == max(listofareas):
          ellipse = cv2.fitEllipse(c)
          cv2.ellipse(anotherempty, ellipse, 	(255,255,255), 2)  
          return anotherempty
  
  
  
  if(mask_yellow.any()):
    
    blurred = cv2.GaussianBlur(mask_yellow, (5, 5), 0)
    imageCanny = cv2.Canny(blurred, 0, 100, 0)
    ret,th = cv2.threshold(imageCanny,127,255, 0)

    #--- Find all the contours in the binary image ---
    contours,hierarchy = cv2.findContours(th,3,1)
    cnt = contours
    big_contour = []
    m = 0
    for i in cnt:

      area = cv2.contourArea(i) #--- find the contour having biggest area ---
      if(area > m):
        m = area
        big_contour = i 

    final = cv2.drawContours(imageCanny, big_contour, -1, (0,255,0), 3)

   

    thresh = cv2.threshold(final, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=2)


    # Find contours, filter using contour threshold area, draw ellipse
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    listofareas =[]
    for c in cnts:
        area = cv2.contourArea(c)
        listofareas.append(area)

    for c in cnts:
        area = cv2.contourArea(c)

        if area == max(listofareas):
          ellipse = cv2.fitEllipse(c)
          cv2.ellipse(anotherempty, ellipse, 	(255,255,255), 2)  
          return anotherempty
  
  if(mask_green.any()):
    
    blurred = cv2.GaussianBlur(mask_green, (5, 5), 0)
    imageCanny = cv2.Canny(blurred, 0, 100, 0)
    ret,th = cv2.threshold(imageCanny,127,255, 0)

    #--- Find all the contours in the binary image ---
    contours,hierarchy = cv2.findContours(th,3,1)
    cnt = contours
    big_contour = []
    m = 0
    for i in cnt:

      area = cv2.contourArea(i) #--- find the contour having biggest area ---
      if(area > m):
        m = area
        big_contour = i 

    final = cv2.drawContours(imageCanny, big_contour, -1, (0,255,0), 3)


    thresh = cv2.threshold(final, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=2)


    # Find contours, filter using contour threshold area, draw ellipse
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    listofareas =[]
    for c in cnts:
        area = cv2.contourArea(c)
        listofareas.append(area)

    for c in cnts:
        area = cv2.contourArea(c)

        if area == max(listofareas):
          ellipse = cv2.fitEllipse(c)
          cv2.ellipse(anotherempty, ellipse, 	(255,255,255), 2)  
          return anotherempty


def extractedClass(measures):
  if(not measures[0]==None):
    return "Abdominal"
  if(not measures[1]==None):
    return "Femur"
  if(measures[2]!=None or measures[3]!=None):
    return "Transthalamique"
  else:
    return "Other"  
  
def convertDicomToPNG(path):
  ds=pydicom.dcmread(path)
  img=ds.pixel_array
  img = np.array(img, dtype = float) 
  img = (img - img.min()) / (img.max() - img.min()) * 255.0  
  img = img.astype(np.uint8)
  return img

## **Get an approximated measure to compare it with real one in order to evaluate the preprocessing**

In [ ]:

def getPredictedMeasureCA(predictedmask,pixel_spacing):
    """
    Abdomen
    input : image, predictedmask and the output of the criteria
    output : image with visualisation
    """
    
    predictedmask = np.where(predictedmask == 255, 3, predictedmask)
    (minorAxis,majorAxis)=getMinorMajorAxis(predictedmask)

    
    predictedMeasureCA=round(pixel_spacing*calculatePerimeter(minorAxis,majorAxis)/20,2) 

    
    return predictedMeasureCA
  
def getPredictedMeasureBIPandCT(predictedmask ,pixel_spacing):
    """
    Brain
    input : image, predictedmask and the output of the criteria
    output : image with visualisation
    """
    
    predictedmask = np.where(predictedmask == 255, 3, predictedmask)
    (minorAxis,majorAxis)=getMinorMajorAxis(predictedmask)
    
    predictedMeasureBIP=round(pixel_spacing*minorAxis/10,2)
    predictedMeasureCT=round(pixel_spacing*calculatePerimeter(minorAxis,majorAxis)/20,2)
    return predictedMeasureCT , predictedMeasureBIP



def extremitiesPoints(mask):
    """
    returns the side extremities  (needs to be replaced with better function)
    input : mask
    output : 2points
    """
    leftX = 50000
    rightX = 0
    for indexy,y in enumerate(mask):
        for indexx,x in enumerate(y):
            if(mask[indexy,indexx].any()):
                if(indexx<leftX):
                    leftX = indexx
                    leftY = indexy
                if(indexx>rightX):
                    rightX = indexx
                    rightY = indexy
    return (leftX,leftY),(rightX,rightY)  


def getDistance(p1,p2):
    dx = (p1[0] - p2[0])
    dy = (p1[1] - p2[1])
    return np.sqrt(dx*dx+dy*dy)


def getPredictedMeasureLF(mask, pixelspacing):
    """
    input :  femur mask its pixelspacing
    output : the 2points of the extremeties
    """

    point1, point2 = extremitiesPoints(mask) ## Replacable by taha's code
    distance = getDistance(point1,point2)
    predictedMeasureLF = round(float(pixelspacing)*distance/10,2)
    
    return predictedMeasureLF

#def femurMeasurements(mask, pixelspacing):
  
    #pixelspacing = [psX,psY]

   # point1, point2 = extremitiesPoints(mask) 
   # predictedMeasureLF = round(getDistance(point1,point2,pixelspacing)/10,2)
    
   # return predictedMeasureLF
  
  
  
  
  
def getMinorMajorAxis(original):
  
    image=original.copy().astype('uint8')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    actualcontours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    actualcontours=removeOutliers(actualcontours,60)
    for i in actualcontours:
       cv2.drawContours(original, i, -1, (0,255,0), 3)
    empty=np.zeros((image.shape[0],image.shape[1],3), np.uint8)
    ConvexHullPoints = contoursConvexHull(actualcontours)
    test41=cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
    imageCannyee = cv2.Canny(test41, 0, 100, 0)
    contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    for cont in contours:
        if len(cont) < 20:
          break
    elps = cv2.fitEllipse(cont)

    anotherempty=np.zeros((image.shape[0],image.shape[1]), np.uint8)
    cv2.ellipse(anotherempty,elps,7,2)
    LISTe=elps[1]

    return LISTe

def contoursConvexHull(contours):
    pts = []
    for i in range(0, len(contours)):
        for j in range(0, len(contours[i])):
            pts.append(contours[i][j])
    pts = np.array(pts)
    result = cv2.convexHull(pts)
    return result

def calculatePerimeter(minorAxis, majorAxis):
    """
    given 2 axis (major and minor), returns the perimeter
    input : major and minor axis length
    output : float (perimeter length)
    """
    t = ((minorAxis-majorAxis)/(minorAxis+majorAxis))**2
    return pi*(minorAxis+majorAxis)*hyp2f1(-0.5, -0.5, 1, t)
